In [1]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy.sparse.linalg import eigsh
from itertools import combinations, product

L = 4  # Number of lattice sites
N_up = 2
N_down = 2
t = 1.0  # Hopping term
U = 4.0  # On-site repulsion

# ---------- Step 1: Generate Basis States ----------
def generate_basis(L, N_up, N_down):
    def bit_repr(indices):
        x = 0
        for i in indices:
            x |= (1 << i)
        return x

    up_states = [bit_repr(c) for c in combinations(range(L), N_up)]
    down_states = [bit_repr(c) for c in combinations(range(L), N_down)]
    
    basis = []
    state_index = {}
    idx = 0
    for up in up_states:
        for down in down_states:
            basis.append((up, down))
            state_index[(up, down)] = idx
            idx += 1
    return basis, state_index

basis, state_index = generate_basis(L, N_up, N_down)
dim = len(basis)

# ---------- Step 2: Construct Hamiltonian ----------
H = dok_matrix((dim, dim), dtype=np.float64)

def count_bits(x):
    return bin(x).count("1")

def apply_hopping(spin_config, other_config, i, j):
    if (spin_config >> i) & 1 and not (spin_config >> j) & 1:
        # move from i to j
        new_spin = spin_config ^ (1 << i) ^ (1 << j)
        phase = (-1) ** (count_bits(spin_config & ((1 << i) - 1)) + count_bits(other_config & ((1 << i) - 1)))
        return new_spin, phase
    return None, None

for idx, (up, down) in enumerate(basis):
    # On-site interaction U * n_up * n_down
    double_occ = up & down
    n_double = count_bits(double_occ)
    H[idx, idx] += U * n_double

    # Hopping for up spins
    for i in range(L):
        j = (i + 1) % L  # periodic boundary
        new_up, phase = apply_hopping(up, down, i, j)
        if new_up is not None:
            new_state = (new_up, down)
            new_idx = state_index.get(new_state)
            if new_idx is not None:
                H[idx, new_idx] += -t * phase
                H[new_idx, idx] += -t * phase  # Hermitian

    # Hopping for down spins
    for i in range(L):
        j = (i + 1) % L
        new_down, phase = apply_hopping(down, up, i, j)
        if new_down is not None:
            new_state = (up, new_down)
            new_idx = state_index.get(new_state)
            if new_idx is not None:
                H[idx, new_idx] += -t * phase
                H[new_idx, idx] += -t * phase

# ---------- Step 3: Diagonalize Hamiltonian ----------
# Compute the ground state eigenvector
eigvals, eigvecs = eigsh(H.tocsr(), k=1, which='SA')  # Smallest algebraic
ground_state = eigvecs[:, 0]
psi_squared = np.abs(ground_state) ** 2

# ---------- Step 4: Print or Save Data ----------
print("Basis State (up, down) -- |ψ|^2:")
for i, (up, down) in enumerate(basis):
    up_str = format(up, f'0{L}b')
    down_str = format(down, f'0{L}b')
    print(f"{up_str} , {down_str} --> {psi_squared[i]:.6f}")

# Optionally save to CSV
import csv
with open("hubbard_1d_data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["up_config", "down_config", "psi_squared"])
    for i, (up, down) in enumerate(basis):
        writer.writerow([format(up, f'0{L}b'), format(down, f'0{L}b'), psi_squared[i]])


Basis State (up, down) -- |ψ|^2:
0011 , 0011 --> 0.000051
0011 , 0101 --> 0.013959
0011 , 1001 --> 0.002807
0011 , 0110 --> 0.002807
0011 , 1010 --> 0.006652
0011 , 1100 --> 0.044610
0101 , 0011 --> 0.013959
0101 , 0101 --> 0.004031
0101 , 1001 --> 0.023667
0101 , 0110 --> 0.023667
0101 , 1010 --> 0.225350
0101 , 1100 --> 0.018242
1001 , 0011 --> 0.002807
1001 , 0101 --> 0.023667
1001 , 1001 --> 0.003289
1001 , 0110 --> 0.069431
1001 , 1010 --> 0.019431
1001 , 1100 --> 0.002093
0110 , 0011 --> 0.002807
0110 , 0101 --> 0.023667
0110 , 1001 --> 0.069431
0110 , 0110 --> 0.003289
0110 , 1010 --> 0.019431
0110 , 1100 --> 0.002093
1010 , 0011 --> 0.006652
1010 , 0101 --> 0.225350
1010 , 1001 --> 0.019431
1010 , 0110 --> 0.019431
1010 , 1010 --> 0.004231
1010 , 1100 --> 0.018314
1100 , 0011 --> 0.044610
1100 , 0101 --> 0.018242
1100 , 1001 --> 0.002093
1100 , 0110 --> 0.002093
1100 , 1010 --> 0.018314
1100 , 1100 --> 0.000000


In [4]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy.sparse.linalg import eigsh
from itertools import combinations
import csv

# Parameters for the 1D Hubbard model
L = 8           # Number of lattice sites
N_up = 4        # Number of up-spin electrons
N_down = 4      # Number of down-spin electrons
t = 1.0         # Hopping parameter
U = 4.0         # On-site interaction strength

# ---------- Step 1: Generate Basis States ----------
def generate_basis(L, N_up, N_down):
    def bit_repr(indices):
        x = 0
        for i in indices:
            x |= (1 << i)
        return x

    up_states = [bit_repr(c) for c in combinations(range(L), N_up)]
    down_states = [bit_repr(c) for c in combinations(range(L), N_down)]

    basis = []
    state_index = {}
    idx = 0
    for up in up_states:
        for down in down_states:
            basis.append((up, down))
            state_index[(up, down)] = idx
            idx += 1
    return basis, state_index

basis, state_index = generate_basis(L, N_up, N_down)
dim = len(basis)
print(f"Basis size: {dim}")

# ---------- Step 2: Construct Hamiltonian ----------
H = dok_matrix((dim, dim), dtype=np.float64)

def count_bits(x):
    return bin(x).count("1")

def apply_hopping(spin_config, other_config, i, j):
    if (spin_config >> i) & 1 and not (spin_config >> j) & 1:
        new_spin = spin_config ^ (1 << i) ^ (1 << j)
        phase = (-1) ** (count_bits(spin_config & ((1 << i) - 1)) +
                         count_bits(other_config & ((1 << i) - 1)))
        return new_spin, phase
    return None, None

for idx, (up, down) in enumerate(basis):
    # On-site interaction U * n_up * n_down
    double_occ = up & down
    n_double = count_bits(double_occ)
    H[idx, idx] += U * n_double

    # Hopping for up-spin electrons
    for i in range(L):
        j = (i + 1) % L  # Periodic boundary
        new_up, phase = apply_hopping(up, down, i, j)
        if new_up is not None:
            new_state = (new_up, down)
            new_idx = state_index.get(new_state)
            if new_idx is not None:
                H[idx, new_idx] += -t * phase
                H[new_idx, idx] += -t * phase  # Hermitian symmetry

    # Hopping for down-spin electrons
    for i in range(L):
        j = (i + 1) % L
        new_down, phase = apply_hopping(down, up, i, j)
        if new_down is not None:
            new_state = (up, new_down)
            new_idx = state_index.get(new_state)
            if new_idx is not None:
                H[idx, new_idx] += -t * phase
                H[new_idx, idx] += -t * phase

# ---------- Step 3: Diagonalize the Hamiltonian ----------
print("Diagonalizing Hamiltonian...")
eigvals, eigvecs = eigsh(H.tocsr(), k=1, which='SA')  # Ground state
ground_state = eigvecs[:, 0]
psi_squared = np.abs(ground_state) ** 2

# ---------- Step 4: Save Data ----------
output_file = "hubbard_1d_L8_data.csv"
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["up_config", "down_config", "psi_squared"])
    for i, (up, down) in enumerate(basis):
        writer.writerow([format(up, f'0{L}b'), format(down, f'0{L}b'), psi_squared[i]])

print(f"Saved data to {output_file}")


Basis size: 4900
Diagonalizing Hamiltonian...
Saved data to hubbard_1d_L8_data.csv


In [59]:
import numpy as np
from openfermion import fermi_hubbard, get_sparse_operator
from scipy.sparse.linalg import eigsh

def generate_hubbard_data(nsites=4, nelectrons=4, U=2.0, t=1.0):
    # Build Hubbard Hamiltonian
    hubbard = fermi_hubbard(1, nsites, tunneling=-t, coulomb=U, periodic=False)
    hamiltonian = get_sparse_operator(hubbard)
    
    # Find ground state
    eigenvalues, eigenvectors = eigsh(hamiltonian, k=1, which='SA')
    psi = eigenvectors[:,0].real
    
    # Calculate |ψ|²
    psi_squared = np.square(np.abs(psi))
    
    # Generate spatial grid
    x = np.linspace(0, nsites-1, len(psi_squared))  # Match grid to psi length
    
    return x, psi_squared

nsites = 4
U_values = [1.0, 2.0, 4.0]
data = {}

for U in U_values:
    x, psi_sq = generate_hubbard_data(nsites=nsites, U=U)
    data[U] = (x, psi_sq)

np.savez("hubbard_data_n6.npz", **{f"U_{U}": np.stack(data[U], axis=0) for U in data})

Epoch 0: Total Loss=0.228836, Data Loss=0.228831, Physics Loss=0.228838
Epoch 1: Total Loss=0.204129, Data Loss=0.204123, Physics Loss=0.204131
Epoch 2: Total Loss=0.183484, Data Loss=0.183478, Physics Loss=0.183487
Epoch 3: Total Loss=0.166694, Data Loss=0.166687, Physics Loss=0.166697
Epoch 4: Total Loss=0.153379, Data Loss=0.153372, Physics Loss=0.153382
Epoch 5: Total Loss=0.143022, Data Loss=0.143015, Physics Loss=0.143026
Epoch 6: Total Loss=0.135075, Data Loss=0.135066, Physics Loss=0.135078
Epoch 7: Total Loss=0.128993, Data Loss=0.128985, Physics Loss=0.128997
Epoch 8: Total Loss=0.124255, Data Loss=0.124245, Physics Loss=0.124259
Epoch 9: Total Loss=0.120375, Data Loss=0.120365, Physics Loss=0.120379
Epoch 10: Total Loss=0.116936, Data Loss=0.116926, Physics Loss=0.116941
Epoch 11: Total Loss=0.113607, Data Loss=0.113596, Physics Loss=0.113612
Epoch 12: Total Loss=0.110154, Data Loss=0.110142, Physics Loss=0.110158
Epoch 13: Total Loss=0.106435, Data Loss=0.106423, Physics Lo